**CONNECT TO GOOGLE DRIVE**

**USE PICKLE to save index:**

In [ ]:
import pickle
def dumpData(d,f):
  fd=open(f,'ab')
  pickle.dump(d,fd)
  fd.close()
def loadData(file):
  fd=open(file,'rb')
  d=pickle.load(fd)
  fd.close()
  return d

In [ ]:
import pandas as pd
import math
import nltk
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy import spatial
import matplotlib.pyplot as plt
nltk.download('punkt')
from nltk import word_tokenize
porter=nltk.PorterStemmer()
filepath='/wiki_movie_plots_deduped.csv'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# df = pd.read_csv('drive/My Drive/semester5/movies2.csv')

df = pd.read_csv(filepath)
df.drop('Wiki Page', axis=1, inplace=True)

In [ ]:

df

# MAKING INVERTED INDEX

In [ ]:
def make_inverted_index(df):
  # df = pd.read_csv(f)
  # df=df['Plot']

  inverted_index={}
  docid=0
  for movie in df:
    tokens=word_tokenize(movie)
    tokens_lower=[w.lower() for w in tokens if w.isalnum()]
    stemmed=[porter.stem(w) for w in tokens_lower]
    for stem in stemmed:
      val = inverted_index.get(stem)
      if val == None:
        l=[1,[(docid,1)]]
        inverted_index[stem]=l
      else:
        l=inverted_index[stem]
        temp =[]
        for i in l[1]:
          temp.append(i[0])
        if docid in temp:
          k=temp.index(docid)
          value=l[1][k][1]+1
          # l[1][k][1]+=1
          l[1][k]=(docid,value)
        if docid not in temp:
          l[1].append((docid,1))
          l[0]+=1
          
      
    docid+=1 
  lentgh=len(inverted_index)
  for i in range(lentgh):
      j=list((inverted_index.keys()))[i]
      frequenvy=inverted_index[j][0]
      idf=math.log2(len(df)/frequenvy)
      if(inverted_index[j][-1]!=idf):
         inverted_index[j].append(idf)
      
  return inverted_index
  # fileath='drive/My Drive/semester5/mynewfile2.csv'

In [ ]:
plotindex=make_inverted_index(df['Plot'])


In [ ]:
# # dumpData(plotindex,'drive/My Drive/semester5/plotindex.pickel')
# plotindex=loadData('drive/My Drive/semester5/plotindex.pickel')
# len(plotindex)

41264

In [ ]:
df['information'] = df['Title'].replace(['unknown','Unknown'],'').astype(str) +' '+ df['Genre'].replace(['unknown','Unknown'],'').astype(str)+' '+df['Director'].replace(['unknown','Unknown'],'').astype(str)+' '+df['Release Year'].replace(['unknown','Unknown'],'').astype(str)

In [ ]:
infoindex=make_inverted_index(df['information'])


In [ ]:
infoindex

# TFIDF CALCULATION AND RANKING:

In [ ]:
tfidf_dictforplot={}
tfidf_listforplot=[]

for term in plotindex.keys():
  tfidf_listforplot.append(term)
  temp=plotindex[term][1]

  for t in temp:
    tfidf_dictforplot[t[0],term]=plotindex[term][2]*(1+math.log2(t[1]))
print((plotindex.keys()))
print(tfidf_listforplot)



tfidf_dictforinfo={}
tfidf_listforinfo=[]

for term in infoindex.keys():
  tfidf_listforinfo.append(term)
  temp=infoindex[term][1]
  
  # print(temp)
  for t in temp:
    tfidf_dictforinfo[t[0],term]=infoindex[term][2]*(1+math.log2(t[1]))
print((infoindex.keys()))
print(tfidf_listforinfo)

dict_keys(['a', 'bartend', 'is', 'work', 'at', 'saloon', 'serv', 'drink', 'to', 'custom', 'after', 'he', 'fill', 'stereotyp', 'irish', 'man', 'bucket', 'with', 'beer', 'carri', 'nation', 'and', 'her', 'follow', 'burst', 'insid', 'they', 'assault', 'the', 'pull', 'hi', 'hat', 'over', 'eye', 'then', 'dump', 'head', 'group', 'begin', 'wreck', 'bar', 'smash', 'fixtur', 'mirror', 'break', 'cash', 'regist', 'spray', 'seltzer', 'water', 'in', 'face', 'befor', 'of', 'policemen', 'appear', 'order', 'everybodi', 'leav', '1', 'moon', 'paint', 'smile', 'hang', 'park', 'night', 'young', 'coupl', 'walk', 'past', 'fenc', 'learn', 'on', 'rail', 'look', 'up', 'embrac', 'get', 'bigger', 'sit', 'down', 'bench', 'by', 'tree', 'view', 'block', 'caus', 'him', 'frown', 'last', 'scene', 'fan', 'woman', 'becaus', 'ha', 'left', 'sky', 'perch', 'shoulder', 'see', 'everyth', 'better', 'film', 'just', 'minut', 'long', 'compos', 'two', 'shot', 'first', 'girl', 'base', 'an', 'altar', 'or', 'tomb', 'hidden', 'from', 

In [ ]:
Dforplot = np.zeros((len(df),len(plotindex)))
for i in tfidf_dictforplot:
  k=tfidf_listforplot.index(i[1])
  Dforplot[i[0]][k] = tfidf_dictforplot[i]
print(Dforplot)

Dforinfo = np.zeros((len(df),len(infoindex)))
for i in tfidf_dictforinfo:
  k=tfidf_listforinfo.index(i[1])
  Dforinfo[i[0]][k] = tfidf_dictforinfo[i]
print(Dforinfo)




In [ ]:
# dumpData(Dforplot,'drive/My Drive/semester5/Dforplot.pickel')
# Dforplot=loadData('drive/My Drive/semester5/Dforplot.pickel')
# print(Dforplot)

[[ 0.09716607 14.71395008  0.1438509  ...  0.          0.
   0.        ]
 [ 0.12331524  0.          0.2877018  ...  0.          0.
   0.        ]
 [ 0.08372355  0.          0.1438509  ...  0.          0.
   0.        ]
 ...
 [ 0.11611224  0.          0.47786234 ...  0.          0.
   0.        ]
 [ 0.19577     0.          0.57540359 ... 13.28771238  0.
   0.        ]
 [ 0.15570393  0.          0.47786234 ...  0.         39.86313714
  13.28771238]]


In [ ]:
def cosine_similarity(query_dict,D,title,tfidf_list):
  if(bool(query_dict)):
   Q=  np.zeros((len(title)))
   res=[]
   for i in title.keys():
     k=tfidf_list.index(i)    
     if i in query_dict.keys():
       Q[k]=query_dict[i]
   for row in range(D.shape[0]):
    result= 1 - spatial.distance.cosine(D[row], Q)
    res.append([row,result])
   retriedved=sorted(res,key=lambda x:x[1],reverse=True)
   retriedved=retriedved[:10]
   top10documentsvectors=[]
   for i in retriedved:
    top10documentsvectors.append(D[i[0]])
   return res,Q,top10documentsvectors
  else:
    return [],[],[]  

**SPELL CORRECTION**

In [ ]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df

# **QUERY PROCESSING **

In [ ]:
from textblob import TextBlob
query=input("Enter your query:-")
def processquery(t):
  tokens=word_tokenize(t)
  tokens_lower=[w.lower() for w in tokens if w.isalnum()]
  correctwords=[]
  #spell correction:
  for i in tokens_lower:
    b = TextBlob(i)
    correctwords.append(str(b.correct()))
  stemmed=[porter.stem(w) for w in correctwords]
  return stemmed

Enter your query:-brother was killed


In [ ]:
query=query.lower()
query=processquery(query)
print(query)

['brother', 'wa', 'kill']


In [ ]:
query_dictforinfo={}
for q in query:
  if q  in infoindex.keys():
     query_dictforinfo[q]=(infoindex[q][2])*(1+math.log2(query.count(q)))
res=cosine_similarity(query_dictforinfo,Dforinfo,infoindex,tfidf_listforinfo)

sorted_resforinfo= sorted(res[0],key=lambda x:x[1],reverse=True)
queryvectorforinfo=res[1]
documentsvectorsforinfo=res[2]

query_dictforplot={}
for q in query:
  if q  in plotindex.keys():
     query_dictforplot[q]=(plotindex[q][2])*(1+math.log2(query.count(q)))
res=cosine_similarity(query_dictforplot,Dforplot,plotindex,tfidf_listforplot)
sorted_resforplot= sorted(res[0],key=lambda x:x[1],reverse=True)
queryvectorforplot=res[1]
documentsvectorsforplot=res[2]
top10=10



print(sorted_resforinfo)
print(sorted_resforplot)

[[9723, 0.34928890260610745], [8644, 0.32272144291467453], [6550, 0.3114584720932485], [1489, 0.29767534012856967], [7969, 0.29317720097356514], [2755, 0.2929947141108672], [2356, 0.29153893674882114], [7434, 0.28541903304083605], [4040, 0.27687020951756236], [2078, 0.2759508008086994], [3298, 0.27187559291352437], [6974, 0.2712487400085659], [5035, 0.2695066480563224], [2869, 0.2621590223259276], [5443, 0.2586070562488998], [6583, 0.2571560139701492], [2756, 0.25535951063254925], [6736, 0.25265192539932424], [3078, 0.24993184198850016], [8329, 0.248620032979309], [1958, 0.2480382732904156], [2727, 0.24573688153314333], [5794, 0.24539427005256242], [4402, 0.24356104493459863], [6599, 0.2428790113088879], [7157, 0.24126351917185995], [8265, 0.24115804843232713], [4523, 0.2394540000640919], [3080, 0.23715889490802788], [5141, 0.23586020164994204], [8605, 0.23581929774085686], [5040, 0.23475677515885818], [116, 0.2346715360296837], [7420, 0.23307490865327762], [5153, 0.2320533204295262], 

In [ ]:
sorted_res=[]
sorted_res.extend(sorted_resforinfo)
sorted_res.extend(sorted_resforplot)
sorted_res=sorted(sorted_res,key=lambda x:x[1],reverse=True)
print(sorted_res)
sorted_res1=[]
for i in sorted_res:
  sorted_res1.append(i[0])
# set1=set(sorted_res1)
# sorted_res=list(set1)
mylist = list(dict.fromkeys(sorted_res1))
print(mylist)

[[9723, 0.34928890260610745], [8644, 0.32272144291467453], [6550, 0.3114584720932485], [1489, 0.29767534012856967], [7969, 0.29317720097356514], [2755, 0.2929947141108672], [2356, 0.29153893674882114], [7434, 0.28541903304083605], [4040, 0.27687020951756236], [2078, 0.2759508008086994], [3298, 0.27187559291352437], [6974, 0.2712487400085659], [5035, 0.2695066480563224], [2869, 0.2621590223259276], [4733, 0.26074376522235454], [5443, 0.2586070562488998], [6583, 0.2571560139701492], [2756, 0.25535951063254925], [6736, 0.25265192539932424], [3078, 0.24993184198850016], [8329, 0.248620032979309], [1958, 0.2480382732904156], [2727, 0.24573688153314333], [5794, 0.24539427005256242], [2593, 0.2439861726698348], [4402, 0.24356104493459863], [6599, 0.2428790113088879], [7157, 0.24126351917185995], [8265, 0.24115804843232713], [4523, 0.2394540000640919], [3080, 0.23715889490802788], [5141, 0.23586020164994204], [8605, 0.23581929774085686], [5040, 0.23475677515885818], [116, 0.2346715360296837], 

In [ ]:
#calculate sorted result::


In [ ]:


pd.set_option('display.max_colwidth', None)
df.iloc[[docid for docid in mylist]].head(top10)




,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Plot
9723,1980,Kill or Be Killed,American,Ivan Hall,"Charlotte Michelle, James Ryan",action,"A famous martial artist, Steve Chase (James Ryan), travels to the desert for what he thinks is an Olympic-style competition. The competition turns out to be a trap set by Baron von Rudloff (Norman Coombes), an ex-Nazi General who is still bitter over the humiliating defeat of his martial arts team at the 1936 Summer Olympics by Japanese martial artist Miyagi (Raymond Ho-Tong). Steve wants to escape when his girlfriend and fellow karateka, Olga (Charlotte Michelle), is deemed unsuitable to continue as part of von Rudloff's team. When Steve and Olga make their escape, von Rudloff sends his sympathetic dwarf henchman Chico (Danie DuPlessis) to travel around the world to recruit the best fighters for his team.\nIn an attempt to get Steve back on his team, von Rudloff sends one of his top fighters, Ruell (Ed Kannemeyer) to kidnap Olga. Steve decides to return but as a member of Miyagi's team. When the tournament begins, Steve finds himself at constant odds with von Rudloff. When von Rudloff forces Steve to face Olga, a plan is set up for the couple to escape. However, Steve and Olga find themselves kidnapped along with nearly all of the karate fighters with the exception of Luke (Douglas Baggott), a monster fighter on von Rudloff's team. Von Rudloff also learns that his henchman Chico was responsible for helping Steve and Olga. When the fighters make their break out of von Rudloff's castle, Steve takes on Luke in the middle of the desert and after defeating him, von Rudloff finds a gun on the ground and knowing that he cannot ever accept defeat, he takes his own life."
8644,1971,Brother John,American,James Goldstone,"Sidney Poitier, Will Geer, Bradford Dillman",unknown,"His arrival into town is clouded by unrest at a factory where workers are seeking to unionize. Local authorities wrongly suspect John to be an outside agitator for the union cause. The suspicions of the local Sheriff and Doc Thomas' son, the District Attorney, grow after they search John's room and find a passport filled with visa stamps from countries all over the world, even some that few Americans are allowed to travel to. They also find newspaper clippings in a variety of different languages. Only Doc Thomas suspects that John's real purpose is something else.\nAfter the funeral of John's sister, he admits to a young woman, Louisa - a teacher at the local elementary school - that his ""work"" is finished, and that he has a few days to ""do nothing"" before he must leave. She decides to initiate a relationship with him, in hopes that he will stay. This puts him at odds with a local man who has had his eyes on her since they were in high school. During his time with Louisa, John mentions that one of his friends, now a union organizer, will die soon. After this happens, word of his prediction finds its way to the Sheriff who uses it as an excuse to arrest him. During his questioning we find out more about his travels, but he will never say exactly what his ""work"" really is. Doc Thomas comes to visit him in jail, and they have a pivotal conversation. John then walks out of jail and leaves town while the Sheriff and his men are preoccupied with the local labor unrest.\nThroughout the film there are allusions to John's true nature in a confrontation with the sheriff, his hesitant relationship with Louisa, his unexplained ability to travel extensively, his apparent facility with multiple languages, his caring aloofness, etc."
6550,1956,Gun Brothers,American,Sidney Salkow,"Buster Crabbe, Ann Robinson",western,"Wyoming, 1879: Chad Santee, a former Cavalry sergeant, is a passenger on a stagecoach bound for Cheyenne, along with saloon singer Rose Fargo and a gambler known as Blackjack Silk.\nChad is on his way to see brother Jubal's ranch for the first time. What he doesn't know is that Jubal is among masked outlaws who rob

# **RELEVENCE FEEDBACK FROM USER**

In [ ]:
print('Enter relevence of the query(0-4')
feedback=[]
if(len(sorted_res)!=0):
 
 for f in range(top10):
  print('Is the document', f ,'relevant')
  x=input(0)
  if(x==''):
    feedback.append(0)
  else:
    x=int(x)
    feedback.append(x)

Enter relevence of the query(0-4
Is the document 0 relevant
Is the document 1 relevant


In [ ]:
feedback

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# **Evaluation based on feedback by user: normalized discounted cummulative gain**

In [ ]:
from sklearn.metrics import dcg_score
import numpy as np
relevance_score = np.asarray([feedback])
print("relevence score:")
print(relevance_score)
true_relevance = np.asarray([sorted(feedback,reverse=True)])
print("True relevence")
print(true_relevance)
dcg = dcg_score(true_relevance, relevance_score)
print("DCG score : ", dcg)
idcg = dcg_score(true_relevance, true_relevance)
print("IDCG score : ", idcg)
if(idcg!=0):
 ndcg = dcg / idcg
 print("nDCG score : ", ndcg)



relevence score:
[[1 1 1 1 1 1 1 1 1 1]]
True relevence
[[1 1 1 1 1 1 1 1 1 1]]
DCG score :  4.543559338088345
IDCG score :  4.543559338088345
nDCG score :  1.0


In [ ]:
feedbackforrelence=[]
for i in feedback:
  if(i==0):
    feedbackforrelence.append(0)
  else:
    feedbackforrelence.append(1)  

# **ROCHHIO ALGORITHM FOR QUERY EXPANSION:**

In [ ]:
inintial_queryvectorforplot=queryvectorforplot
print(inintial_queryvectorforplot)
print(documentsvectorsforplot)


In [ ]:
#centroid of set of all relevent documents:
releventdocumentsforplot=[]
nonreleventdocumentsforplot=[]
for f in feedbackforrelence:
  if(f==0):
    index=feedbackforrelence.index(f)
    nonreleventdocumentsforplot.append(documentsvectorsforplot[index])
  if(f==1):
        index=feedbackforrelence.index(f)
        releventdocumentsforplot.append(documentsvectorsforplot[index])


print(len(nonreleventdocumentsforplot))
print(len(releventdocumentsforplot))

In [ ]:
inintial_queryvectorforinfo=queryvectorforinfo
print(inintial_queryvectorforinfo)
print(documentsvectorsforinfo)


[0. 0. 0. ... 0. 0. 0.]
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]


In [ ]:
#centroid of set of all relevent documents:
releventdocumentsforinfo=[]
nonreleventdocumentsforinfo=[]
for f in feedbackforrelence:
  if(f==0):
    index=feedbackforrelence.index(f)
    nonreleventdocumentsforinfo.append(documentsvectorsforinfo[index])
  if(f==1):
        index=feedbackforrelence.index(f)
        releventdocumentsforinfo.append(documentsvectorsforinfo[index])


print(len(nonreleventdocumentsforinfo))
print(len(releventdocumentsforinfo))

2
8


In [ ]:
# print(releventdocuments)
# releventdocuments=np.array(releventdocuments)
# print(releventdocuments)
# print(nonreleventdocuments)
nonreleventdocumentsforplot=np.array(nonreleventdocumentsforplot)
print(nonreleventdocumentsforplot)
k=np.zeros(len(plotindex))
for i in nonreleventdocumentsforplot:
  k=i+k

k=k/len(nonreleventdocumentsforplot)
nonrelCforplot=k


releventdocumentsforplot=np.array(releventdocumentsforplot)
print(releventdocumentsforplot)
k=np.zeros(len(plotindex))
for i in releventdocumentsforplot:
  k=i+k

k=k/len(releventdocumentsforplot)
relCforplot=k
# nonreleventdocuments.

[[0.08372355 0.         0.1438509  ... 0.         0.         0.        ]
 [0.08372355 0.         0.1438509  ... 0.         0.         0.        ]]
[[0.        0.        0.1438509 ... 0.        0.        0.       ]
 [0.        0.        0.1438509 ... 0.        0.        0.       ]
 [0.        0.        0.1438509 ... 0.        0.        0.       ]
 ...
 [0.        0.        0.1438509 ... 0.        0.        0.       ]
 [0.        0.        0.1438509 ... 0.        0.        0.       ]
 [0.        0.        0.1438509 ... 0.        0.        0.       ]]


In [ ]:
nonreleventdocumentsforinfo=np.array(nonreleventdocumentsforinfo)
print(nonreleventdocumentsforinfo)
k=np.zeros(len(infoindex))
for i in nonreleventdocumentsforinfo:
  k=i+k

k=k/len(nonreleventdocumentsforinfo)
nonrelCforinfo=k


releventdocumentsforinfo=np.array(releventdocumentsforinfo)
print(releventdocumentsforinfo)
k=np.zeros(len(infoindex))
for i in releventdocumentsforinfo:
  k=i+k

k=k/len(releventdocumentsforinfo)
relCforinfo=k
# nonreleventdocuments.

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
newqueryforinfo=2*inintial_queryvectorforinfo+0.9*relCforinfo-0.5*nonrelCforinfo
print(newqueryforinfo)
newqueryforplot=2*inintial_queryvectorforplot+0.9*relCforplot-0.5*nonrelCforplot
print(newqueryforplot)

[0. 0. 0. ... 0. 0. 0.]
[-0.04186177  0.          0.05754036 ...  0.          0.
  0.        ]


In [ ]:
resuforinfo=[]
for row in range(Dforinfo.shape[0]):
    result= 1 - spatial.distance.cosine(Dforinfo[row], newqueryforinfo)
    resuforinfo.append([row,result])
sorted_resu= sorted(resuforinfo,key=lambda x:x[1],reverse=True) 

In [ ]:
resuforplot=[]
for row in range(Dforplot.shape[0]):
    result= 1 - spatial.distance.cosine(Dforplot[row], newqueryforplot)
    resuforplot.append([row,result])
sorted_resu= sorted(resuforplot,key=lambda x:x[1],reverse=True) 

In [ ]:
resrochio=[]
resrochio.extend(resuforplot)
resrochio.extend(resuforinfo)
resrochio=sorted(resrochio,key=lambda x:x[1],reverse=True)
print(resrochio)
resrochio1=[]
for i in resrochio:
  resrochio1.append(i[0])
resrochio2 = list(dict.fromkeys(resrochio1))
print(resrochio2)

[[8903, 0.8987865229798349], [3700, 0.8289071056836324], [750, 0.40005973887709234], [3883, 0.38114582495676974], [303, 0.36819148637544585], [1049, 0.3578437111155227], [3467, 0.3533700196296409], [6947, 0.3513303814219948], [1728, 0.3469191875541562], [1439, 0.3446418540754467], [7935, 0.34410125688783655], [1813, 0.3439805922525977], [5770, 0.34230506574790054], [5276, 0.3322923083262692], [1621, 0.3264025032726081], [621, 0.31118645097793696], [1050, 0.30886593210475133], [34, 0.30865771940222086], [7983, 0.28654565977746427], [8043, 0.2849372873144975], [3942, 0.2719953459296942], [8162, 0.26854614386382836], [8393, 0.26830751267861164], [9721, 0.26577023353177487], [7683, 0.2645970469038621], [8751, 0.26048545443363613], [1550, 0.25764267963280385], [2414, 0.25061455919886755], [5347, 0.24867884102295645], [3204, 0.24768413503185638], [1136, 0.24742078531928668], [2999, 0.24399372831003385], [687, 0.24266987941196216], [2949, 0.2414511654157928], [8508, 0.2392699313499359], [4008

In [ ]:
df = pd.read_csv(filepath,nrows=10000)

In [ ]:
pd.set_option('display.max_colwidth', None)
df.iloc[[docid for docid in resrochio2]].head(top10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
8903,1973,Cops and Robbers,American,Aram Avakian,"Joseph Bologna, Cliff Gorman","crime, comedy",https://en.wikipedia.org/wiki/Cops_and_Robbers_(1973_film),"Tom and Joe are disillusioned New York City cops who decide to pull off one big robbery in their uniforms and retire on the proceeds. Bearer bonds are what the local Mafia boss will buy, so that's what they go for. Central Park is the location for the switch. Can they come out of it with their lives and the money?"
3700,1943,What a Woman!,American,Irving Cummings,"Rosalind Russell, Brian Aherne",comedy,https://en.wikipedia.org/wiki/What_a_Woman!,"Henry Pepper (Brian Aherne), top writer for Knickerbocker magazine, is assigned to write a profile on Carol Ainsley (Rosalind Russell), who has been named the outstanding career woman of the year. Carol, a super agent and star-maker, has just scooped her competition by selling the movie rights to the romance novel Whirlwind and is spending a fortune to find the perfect actor to play the male lead. When Carol learns that the book's author, Anthony Street, may be the man to play his own hero, she searches him out and discovers that he is actually Professor Michael Cobb (Willard Parker) of Buxton College.\nAlthough handsome and blonde, the professor is an intellectual snob immersed in Elizabethan literature, and consequently, is horrified when he is exposed as the writer of a romance novel. While at Buxton, Carol gets Michael in trouble with the faculty and convinces him to accompany her to New York. There she takes over his life, arranging for lessons in comportment and charm. Michael is a failure at speaking the romantic words he wrote, however, and after his screen test proves a dismal failure, he decides to return to Buxton.\nHenry, meanwhile, has become intrigued by Carol and has decided that she would be terrific if she developed her human side more. Intending to see if she has anything other than a dollar sign for a heart, Henry contacts Michael and convinces the professor that he is in love with Carol. While radiating the charm and assurance that Carol has taught him, Michael begins to court her. Their courtship becomes headline news, and although she is not in love with him, Carol is afraid to tell him the truth for fear that he might walk out on his contract.\nHenry is thoroughly enjoying Carol's predicament until he kisses her and begins to fall in love with her himself. When Carol tries to trick Michael into going to Hollywood while she takes refuge at her father's house in Washington, D.C., Michael outsmarts her, follows her home and announces their engagement. Thus trapped, Carol agrees to the marriage.\nOn the eve of the wedding, the guests are socializing in the various rooms of the Ainsley house when Carol, angry at Henry for agreeing to be the best man, goes to his room to confront him. After Henry insults Carol and accuses her of being only a ""ten percent woman,"" she slaps him, runs into the hallway and announces that she is calling off the wedding because she is not in love with Michael and refuses to be married just for the sake of business. Henry listens to her speech in admiration, and when she finishes, she rushes into his arms.[1]"
750,1928,What a Night!,American,A. Edward Sutherland,"Bebe Daniels, Neil Hamilton, William Austin",romantic comedy,https://en.wikipedia.org/wiki/What_a_Night!_(1928_film),"Daniels, in her final silent film,[3] plays Dorothy Winston, an heiress who sets out to become a newspaper reporter and, when she breaks a big story, finds herself in peril.[4]"
3883,1944,"Oh, What a Night",American,William Beaudine,"Jean Parker, Edmund Lowe",comedy,"https://en.wikipedia.org/wiki/Oh,_What_a_Night_(1944_film)",A man is keeping a secret from his young niece: he is an international jewel thief.
303,1919,What Every Woman Learns,American,Fred Niblo,"Enid Bennett, Milton Sills",drama,https://en.wikipedia.org/wiki/What_Every_Woman_